# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-524a517ea7-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and composer explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the composers for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for composer

3. Identify the BGP for film genre

4. Find how many films are been released in the United States of America from 2010-01-01 to 2015-31-12 for each film genre available (the result set must be genre IRI, label and #films).

5. Consider the timespan from 2001-01-01 to nowadays. Find the number of films released in this timespan divided by film genre and return only those with more than 50 films released every year (the result set must be genre IRI and label).

6. Consider the composers of Western films and their country of citizenship. Count the number of Western Films on which worked a composer grouped by country of citizenship and return the top-10 (the result set must be country IRI, label and #films).. 

7. Consider the decades from 1961 to 1970, and from 2001 to 2010 and compare the total number of western films released in each one (the result set must be two couples decade and #western films of that decade).

8. Consider the decades from 1961 to 1970, and from 2001 to 2010 and select only western films. Then, for each decade, compare the average cast members size per film (the result set must be two couples decade and average cast members size of that decade).

## Task 1
Identify the BGP for films

In [10]:
# query example
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind properties of Woody Allen
   wd:Q25089 ?p ?obj .
FILTER(!isLiteral(?obj)) .
   # get the labels 
   ?p sc:name ?pname.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('pname', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1283'), ('pname', 'filmography')]
[('p', 'http://www.wikidata.org/prop/direct/P1303'), ('pname', 'instrument')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1455'), ('pname', 'list of works')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http:/

In [11]:
# Trying to find filmography objects
# wd:Q25089 --> 'Woody Allen'
# wdth:P1283 --> 'filmography'
# query example
queryString = """
SELECT ?obj ?objname
WHERE {
   # bind properties
   wd:Q25089 wdt:P1283 ?obj .
   # get the labels
   ?obj sc:name ?objname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2455835'), ('objname', 'Woody Allen filmography')]
1


In [12]:
# Trying to find Woody Allen filmography properties
# Q2455835 --> 'Woody Allen filmography'
queryString = """
SELECT DISTINCT ?p ?name ?o ?objname
WHERE {
   #bind the properties
   wd:Q2455835 ?p ?o . 
   # get the labels
   ?p sc:name ?name . 
   ?o sc:name ?objname .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject'), ('o', 'http://www.wikidata.org/entity/Q25089'), ('objname', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('objname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1371849'), ('objname', 'filmography')]
3


In [13]:
# trying to find films properties
# Q11424 --> 'film' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?objname
WHERE {
   wd:Q11424 ?p ?o .
   ?p sc:name ?pname .
   ?o sc:name ?objname .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pname', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q1660187'), ('objname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P345'), ('objname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1040'), ('objname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P344'), ('objname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q17738'), ('objname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q55321'), ('objname', 'Ca

In [15]:
# trying to find films properties
# Q11424 --> 'film' 
queryString = """
SELECT DISTINCT ?o ?objname
WHERE {
   VALUES ?p {wdt:P1963} #changed brackets
   wd:Q11424 ?p ?o .
   ?o sc:name ?objname 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P345'), ('objname', 'IMDb ID')]
[('o', 'http://www.wikidata.org/entity/P1040'), ('objname', 'film editor')]
[('o', 'http://www.wikidata.org/entity/P344'), ('objname', 'director of photography')]
[('o', 'http://www.wikidata.org/entity/P1476'), ('objname', 'title')]
[('o', 'http://www.wikidata.org/entity/P2047'), ('objname', 'duration')]
[('o', 'http://www.wikidata.org/entity/P162'), ('objname', 'producer')]
[('o', 'http://www.wikidata.org/entity/P921'), ('objname', 'main subject')]
[('o', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
[('o', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
[('o', 'http://www.wikidata.org/entity/P3816'), ('objname', 'film script')]
10


Final query for this task

In [16]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?objname
WHERE {
   VALUES ?p {wdt:P1963} #changed brackets
   wd:Q11424 ?p ?o .
   ?o sc:name ?objname 
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P345'), ('objname', 'IMDb ID')]
[('o', 'http://www.wikidata.org/entity/P1040'), ('objname', 'film editor')]
[('o', 'http://www.wikidata.org/entity/P344'), ('objname', 'director of photography')]
[('o', 'http://www.wikidata.org/entity/P1476'), ('objname', 'title')]
[('o', 'http://www.wikidata.org/entity/P2047'), ('objname', 'duration')]
[('o', 'http://www.wikidata.org/entity/P162'), ('objname', 'producer')]
[('o', 'http://www.wikidata.org/entity/P921'), ('objname', 'main subject')]
[('o', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
[('o', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
[('o', 'http://www.wikidata.org/entity/P3816'), ('objname', 'film script')]
[('o', 'http://www.wikidata.org/entity/P179'), ('objname', 'part of the series')]
[('o', 'http://www.wikidata.org/entity/P480'), ('objname', 'FilmAffinity ID')]
[('o', 'http://www.wikidata.org/entity/P136'), ('objname', 'genre')]


## Task 2
Identify the BGP for composer

In [18]:
# write your queries
# P86 --> 'composer'
queryString = """
SELECT DISTINCT ?s ?sname ?o ?objname
WHERE {
   ?s wdt:P86 ?o ;  
      sc:name ?sname .
   ?o sc:name ?objname 
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q22909117'), ('sname', 'Stroke'), ('o', 'http://www.wikidata.org/entity/Q293218'), ('objname', 'Joan Tower')]
[('s', 'http://www.wikidata.org/entity/Q13657577'), ('sname', 'Heartbeats'), ('o', 'http://www.wikidata.org/entity/Q2682478'), ('objname', 'Tom Kestens')]
[('s', 'http://www.wikidata.org/entity/Q13693967'), ('sname', 'Cross'), ('o', 'http://www.wikidata.org/entity/Q3866310'), ('objname', 'Paul Wong')]
[('s', 'http://www.wikidata.org/entity/Q13647258'), ('sname', 'Marupakkam'), ('o', 'http://www.wikidata.org/entity/Q6456674'), ('objname', 'L. Vaidyanathan')]
[('s', 'http://www.wikidata.org/entity/Q48671658'), ('sname', 'Love Maadi Nodu'), ('o', 'http://www.wikidata.org/entity/Q6456674'), ('objname', 'L. Vaidyanathan')]
[('s', 'http://www.wikidata.org/entity/Q15586278'), ('sname', 'Obliehanie Bystrice'), ('o', 'http://www.wikidata.org/entity/Q692171'), ('objname', 'Ján Cikker')]
[('s', 'http://www.wikidata.org/entity/Q15723755'), ('s

Final query for this task

In [19]:
# write your final query
# P86 --> 'composer'
queryString = """
SELECT DISTINCT ?composition ?composition_name ?o ?composername
WHERE {
   ?composition wdt:P86 ?o ;  
      sc:name ?composition_name .
   ?o sc:name ?composername 
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('composition', 'http://www.wikidata.org/entity/Q22909117'), ('composition_name', 'Stroke'), ('o', 'http://www.wikidata.org/entity/Q293218'), ('composername', 'Joan Tower')]
[('composition', 'http://www.wikidata.org/entity/Q13657577'), ('composition_name', 'Heartbeats'), ('o', 'http://www.wikidata.org/entity/Q2682478'), ('composername', 'Tom Kestens')]
[('composition', 'http://www.wikidata.org/entity/Q13693967'), ('composition_name', 'Cross'), ('o', 'http://www.wikidata.org/entity/Q3866310'), ('composername', 'Paul Wong')]
[('composition', 'http://www.wikidata.org/entity/Q13647258'), ('composition_name', 'Marupakkam'), ('o', 'http://www.wikidata.org/entity/Q6456674'), ('composername', 'L. Vaidyanathan')]
[('composition', 'http://www.wikidata.org/entity/Q48671658'), ('composition_name', 'Love Maadi Nodu'), ('o', 'http://www.wikidata.org/entity/Q6456674'), ('composername', 'L. Vaidyanathan')]
[('composition', 'http://www.wikidata.org/entity/Q15586278'), ('composition_name', 'Obl

## Task 3
Identify the BGP for film genre

In [25]:
# write your queries
# P136 --> genre
queryString = """
SELECT ?s ?sname ?o ?objname
WHERE {
    ?s wdt:P136 ?o ;
       sc:name ?sname .
    ?o sc:name ?objname
}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8204826'), ('sname', 'Harpist of Keros'), ('o', 'http://www.wikidata.org/entity/Q2586345'), ('objname', 'Cycladic art')]
[('s', 'http://www.wikidata.org/entity/Q8195511'), ('sname', 'Ayinde Barrister'), ('o', 'http://www.wikidata.org/entity/Q3509353'), ('objname', 'fuji music')]
[('s', 'http://www.wikidata.org/entity/Q15730544'), ('sname', 'Crying for No Reason'), ('o', 'http://www.wikidata.org/entity/Q1987980'), ('objname', 'breakstep')]
[('s', 'http://www.wikidata.org/entity/Q15732538'), ('sname', 'Chapliniana'), ('o', 'http://www.wikidata.org/entity/Q4484381'), ('objname', 'ballet film')]
[('s', 'http://www.wikidata.org/entity/Q15731615'), ('sname', 'Scribblenauts Unmasked: A DC Comics Adventure'), ('o', 'http://www.wikidata.org/entity/Q5370763'), ('objname', 'emergent gameplay')]
[('s', 'http://www.wikidata.org/entity/Q104891471'), ('sname', 'Ajin'), ('o', 'http://www.wikidata.org/entity/Q61942616'), ('objname', 'supernatural anime an

In [39]:
# Q4484381 --> 'ballet film'
# P136 --> genre
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    wd:Q4484381 ?p ?o  .
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q201658'), ('objname', 'film genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q41425'), ('objname', 'ballet')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('objname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q21934619'), ('objname', 'Category:Ballet films')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pname', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q4453959'), ('objname', 'television ballet\u200e')]
5


In [40]:
# Q201658 --> 'film genre'
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    wd:Q201658 ?p ?o  .
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1792379'), ('objname', 'art genre')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q5817098'), ('objname', 'Category:Film genres')]
[('p', 'http://www.wikidata.org/prop/direct/P8225'), ('pname', 'is metaclass for'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('objname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q590870'), ('objname', 'cinematography')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q19361238'), ('objname', 'Wikidata metaclass')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6621591'), ('objname', 'list of genres')]
6


In [4]:
# Q5817098 --> 'Category:Film genres'
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    wd:Q5817098 ?p ?o  .
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q201658'), ('objname', 'film genre')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4167836'), ('objname', 'Wikimedia category')]
2


Final query for this task

In [5]:
# write your final query
# Q201658 --> 'film genre'
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    wd:Q201658 ?p ?o  .
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1792379'), ('objname', 'art genre')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q5817098'), ('objname', 'Category:Film genres')]
[('p', 'http://www.wikidata.org/prop/direct/P8225'), ('pname', 'is metaclass for'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('objname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q590870'), ('objname', 'cinematography')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q19361238'), ('objname', 'Wikidata metaclass')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6621591'), ('objname', 'list of genres')]
6


## Task 4
Find how many films are been released in the United States of America from 2010-01-01 to 2015-31-12 for each film genre available (the result set must be genre IRI, label and #films).

In [35]:
# write your queries
# Q11424 --> 'film'
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    wd:Q11424 ?p ?o  .
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pname', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q1660187'), ('objname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P345'), ('objname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1040'), ('objname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P344'), ('objname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q17738'), ('objname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q55321'), ('objname', 'Ca

In [8]:
# write your queries
# Q11424 --> 'film'
# P31 --> 'instance of'
# P495 --> 'country of origin'
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P495 ?o.
    ?p sc:name ?pname .
   ?o sc:name ?objname 
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q6466199'), ('pname', 'Love Child'), ('o', 'http://www.wikidata.org/entity/Q786'), ('objname', 'Dominican Republic')]
[('p', 'http://www.wikidata.org/entity/Q30314457'), ('pname', 'Nashid al-Amal'), ('o', 'http://www.wikidata.org/entity/Q124943'), ('objname', 'Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/entity/Q27051825'), ('pname', 'The Desert of the Desert'), ('o', 'http://www.wikidata.org/entity/Q40362'), ('objname', 'Sahrawi Arab Democratic Republic')]
[('p', 'http://www.wikidata.org/entity/Q17492410'), ('pname', 'For the Best and for the Onion'), ('o', 'http://www.wikidata.org/entity/Q1032'), ('objname', 'Niger')]
[('p', 'http://www.wikidata.org/entity/Q17059721'), ('pname', 'Mr and Mrs'), ('o', 'http://www.wikidata.org/entity/Q1033'), ('objname', 'Nigeria')]
[('p', 'http://www.wikidata.org/entity/Q21010846'), ('pname', '93 Days'), ('o', 'http://www.wikidata.org/entity/Q1033'), ('objname', 'Nigeria')]
[('p', 'http://www.wikida

In [98]:
# write your queries
# Q11424 --> 'film'
# P31 --> 'instance of'
# P495 --> 'country of origin'
# Q30 -->'United States of America'
#Restricting data to USA only
queryString = """
SELECT ?p ?pname ?o ?objname 
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P495 ?o.
    ?p sc:name ?pname .
   ?o sc:name ?objname. 
   filter(?o = wd:Q30)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13677566'), ('pname', 'Timberjack'), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q15490985'), ('pname', 'Jailhouse Heat'), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q15046558'), ('pname', 'Life After Beth'), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q15584559'), ('pname', 'Salvation Nell'), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q15596853'), ('pname', 'Burying the Ex'), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q15728077'), ('pname', "We're Only Human"), ('o', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of Am

In [101]:
# Q11424 --> 'film'
# P31 --> 'instance of'
# P495 --> 'country of origin'
# Q30 -->'United States of America'
# P136 --> 'genre'
#Grouping data wrt genre
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?gen ?GenName
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P495 ?o;
    wdt:P136 ?gen.
    ?p sc:name ?pname .
   ?o sc:name ?objname. 
   ?gen sc:name ?GenName.
   filter(?o = wd:Q30)
}
GROUP BY ?gen ?GenName
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '16108'), ('gen', 'http://www.wikidata.org/entity/Q130232'), ('GenName', 'drama')]
[('NumOfFilms', '4913'), ('gen', 'http://www.wikidata.org/entity/Q93204'), ('GenName', 'documentary film')]
[('NumOfFilms', '115'), ('gen', 'http://www.wikidata.org/entity/Q3272147'), ('GenName', 'found footage')]
[('NumOfFilms', '647'), ('gen', 'http://www.wikidata.org/entity/Q224700'), ('GenName', 'comedy horror')]
[('NumOfFilms', '121'), ('gen', 'http://www.wikidata.org/entity/Q909586'), ('GenName', 'splatter film')]
[('NumOfFilms', '6'), ('gen', 'http://www.wikidata.org/entity/Q222926'), ('GenName', 'comedy of manners')]
[('NumOfFilms', '2'), ('gen', 'http://www.wikidata.org/entity/Q858784'), ('GenName', 'mecha')]
[('NumOfFilms', '7'), ('gen', 'http://www.wikidata.org/entity/Q1257444'), ('GenName', 'film adaptation')]
[('NumOfFilms', '7'), ('gen', 'http://www.wikidata.org/entity/Q24862'), ('GenName', 'short film')]
[('NumOfFilms', '22'), ('gen', 'http://www.wikidata.org/entity

In [114]:
#Restricting released period
# P577 --> 'publication date'
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?gen ?GenName
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P495 ?o;
    wdt:P136 ?gen;
    wdt:P577 ?Pdate.
    ?p sc:name ?pname .
   ?o sc:name ?objname. 
   ?gen sc:name ?GenName.
   filter(?o = wd:Q30)
   FILTER(?Pdate > "2010"^^xsd:gYear && ?Pdate < "2015"^^xsd:gYear)
}
GROUP BY ?gen ?GenName
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1962'), ('gen', 'http://www.wikidata.org/entity/Q130232'), ('GenName', 'drama')]
[('NumOfFilms', '1077'), ('gen', 'http://www.wikidata.org/entity/Q93204'), ('GenName', 'documentary film')]
[('NumOfFilms', '135'), ('gen', 'http://www.wikidata.org/entity/Q224700'), ('GenName', 'comedy horror')]
[('NumOfFilms', '6'), ('gen', 'http://www.wikidata.org/entity/Q21010853'), ('GenName', 'drama')]
[('NumOfFilms', '5'), ('gen', 'http://www.wikidata.org/entity/Q3072042'), ('GenName', 'kung fu film')]
[('NumOfFilms', '24'), ('gen', 'http://www.wikidata.org/entity/Q909586'), ('GenName', 'splatter film')]
[('NumOfFilms', '105'), ('gen', 'http://www.wikidata.org/entity/Q3272147'), ('GenName', 'found footage')]
[('NumOfFilms', '11'), ('gen', 'http://www.wikidata.org/entity/Q8812380'), ('GenName', 'pseudo-documentary')]
[('NumOfFilms', '1'), ('gen', 'http://www.wikidata.org/entity/Q9503'), ('GenName', 'suspense')]
[('NumOfFilms', '1'), ('gen', 'http://www.wikidata.org/entity/Q17

Final query for this task

In [117]:
# write your final query 
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?gen ?GenName
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P495 ?o;
    wdt:P136 ?gen;
    wdt:P577 ?Pdate.
    ?p sc:name ?pname .
   ?o sc:name ?objname. 
   ?gen sc:name ?GenName.
   filter(?o = wd:Q30)
   FILTER(?Pdate > "2010"^^xsd:gYear && ?Pdate < "2015"^^xsd:gYear)
}
GROUP BY ?gen ?GenName
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1962'), ('gen', 'http://www.wikidata.org/entity/Q130232'), ('GenName', 'drama')]
[('NumOfFilms', '1077'), ('gen', 'http://www.wikidata.org/entity/Q93204'), ('GenName', 'documentary film')]
[('NumOfFilms', '135'), ('gen', 'http://www.wikidata.org/entity/Q224700'), ('GenName', 'comedy horror')]
[('NumOfFilms', '6'), ('gen', 'http://www.wikidata.org/entity/Q21010853'), ('GenName', 'drama')]
[('NumOfFilms', '5'), ('gen', 'http://www.wikidata.org/entity/Q3072042'), ('GenName', 'kung fu film')]
[('NumOfFilms', '24'), ('gen', 'http://www.wikidata.org/entity/Q909586'), ('GenName', 'splatter film')]
[('NumOfFilms', '105'), ('gen', 'http://www.wikidata.org/entity/Q3272147'), ('GenName', 'found footage')]
[('NumOfFilms', '11'), ('gen', 'http://www.wikidata.org/entity/Q8812380'), ('GenName', 'pseudo-documentary')]
[('NumOfFilms', '1'), ('gen', 'http://www.wikidata.org/entity/Q9503'), ('GenName', 'suspense')]
[('NumOfFilms', '1'), ('gen', 'http://www.wikidata.org/entity/Q17

## Task 5
Consider the timespan from 2001-01-01 to nowadays. Find the number of films released in this timespan divided by film genre and return only those with more than 50 films released every year (the result set must be genre IRI and label).

In [128]:
# write your queries
#Grouping data wrt genre
queryString = """
SELECT distinct (COUNT(?p) as ?NumOfFilms) ?gen ?GenName
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P136 ?gen;
    wdt:P577 ?Pdate.

   ?gen sc:name ?GenName.
   
   FILTER(?Pdate > "2001-01-01"^^xsd:dateTime)
}
GROUP BY ?gen ?GenName
having(Count(?p) > 50)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '116'), ('gen', 'http://www.wikidata.org/entity/Q21010853'), ('GenName', 'drama')]
[('NumOfFilms', '29110'), ('gen', 'http://www.wikidata.org/entity/Q130232'), ('GenName', 'drama')]
[('NumOfFilms', '18932'), ('gen', 'http://www.wikidata.org/entity/Q93204'), ('GenName', 'documentary film')]
[('NumOfFilms', '704'), ('gen', 'http://www.wikidata.org/entity/Q224700'), ('GenName', 'comedy horror')]
[('NumOfFilms', '228'), ('gen', 'http://www.wikidata.org/entity/Q3272147'), ('GenName', 'found footage')]
[('NumOfFilms', '67'), ('gen', 'http://www.wikidata.org/entity/Q1760864'), ('GenName', 'nature documentary')]
[('NumOfFilms', '121'), ('gen', 'http://www.wikidata.org/entity/Q1257444'), ('GenName', 'film adaptation')]
[('NumOfFilms', '202'), ('gen', 'http://www.wikidata.org/entity/Q909586'), ('GenName', 'splatter film')]
[('NumOfFilms', '276'), ('gen', 'http://www.wikidata.org/entity/Q1341051'), ('GenName', 'post-apocalyptic film')]
[('NumOfFilms', '2885'), ('gen', 'htt

Final query for this task

In [129]:
# write your final query
#Grouping data wrt genre
queryString = """
SELECT distinct (COUNT(?p) as ?NumOfFilms) ?gen ?GenName
WHERE {
    ?p wdt:P31 wd:Q11424;
    wdt:P136 ?gen;
    wdt:P577 ?Pdate.

   ?gen sc:name ?GenName.
   
   FILTER(?Pdate > "2001-01-01"^^xsd:dateTime)
}
GROUP BY ?gen ?GenName
having(Count(?p) > 50)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '116'), ('gen', 'http://www.wikidata.org/entity/Q21010853'), ('GenName', 'drama')]
[('NumOfFilms', '29110'), ('gen', 'http://www.wikidata.org/entity/Q130232'), ('GenName', 'drama')]
[('NumOfFilms', '18932'), ('gen', 'http://www.wikidata.org/entity/Q93204'), ('GenName', 'documentary film')]
[('NumOfFilms', '704'), ('gen', 'http://www.wikidata.org/entity/Q224700'), ('GenName', 'comedy horror')]
[('NumOfFilms', '228'), ('gen', 'http://www.wikidata.org/entity/Q3272147'), ('GenName', 'found footage')]
[('NumOfFilms', '67'), ('gen', 'http://www.wikidata.org/entity/Q1760864'), ('GenName', 'nature documentary')]
[('NumOfFilms', '121'), ('gen', 'http://www.wikidata.org/entity/Q1257444'), ('GenName', 'film adaptation')]
[('NumOfFilms', '202'), ('gen', 'http://www.wikidata.org/entity/Q909586'), ('GenName', 'splatter film')]
[('NumOfFilms', '276'), ('gen', 'http://www.wikidata.org/entity/Q1341051'), ('GenName', 'post-apocalyptic film')]
[('NumOfFilms', '2885'), ('gen', 'htt

## Task 6
Consider the composers of Western films and their country of citizenship. Count the number of Western Films on which worked a composer grouped by country of citizenship and return the top-10 (the result set must be country IRI, label and #films)..

In [136]:
# write your queries
# write your queries
# Q11424 --> 'film'
queryString = """
SELECT ?p ?pname ?gen ?genre 
WHERE {
   ?p wdt:P31 wd:Q11424;
    wdt:P136 ?gen.
    ?p sc:name ?pname .
   ?gen sc:name ?genre .
   
   Filter(Regex(?genre , "[Ww]estern film"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q15882802'), ('pname', 'The Storm'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.org/entity/Q16200305'), ('pname', 'Fangs of the Arctic'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.org/entity/Q17011616'), ('pname', 'Mr. Texas'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.org/entity/Q17036710'), ('pname', 'The Drifting Avenger'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.org/entity/Q17052173'), ('pname', 'The Desert of the Lost'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.org/entity/Q17053125'), ('pname', 'Between Dangers'), ('gen', 'http://www.wikidata.org/entity/Q172980'), ('genre', 'Western film')]
[('p', 'http://www.wikidata.

In [140]:
# write your queries
# Q11424 --> 'film'
# Q172980 --> 'Western film'
# P86 --> 'composer' From the above query.
# P27 --> 'nationality' from the given data.
queryString = """
SELECT ?p ?pname ?s ?sname ?composer_name ?citizenship
WHERE {
   ?s ?p wd:Q172980;
   wdt:P86 ?cname.
   ?cname wdt:P27 ?coun.
    
    ?s sc:name ?sname .
   ?p sc:name ?pname .
   ?cname sc:name ?composer_name.
   ?coun sc:name ?citizenship.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('s', 'http://www.wikidata.org/entity/Q1305262'), ('sname', 'Custer of the West'), ('composer_name', 'Bernardo Segall'), ('citizenship', 'Brazilians')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('s', 'http://www.wikidata.org/entity/Q949228'), ('sname', 'Sholay'), ('composer_name', 'Rahul Dev Burman'), ('citizenship', 'British India')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('s', 'http://www.wikidata.org/entity/Q6402614'), ('sname', 'Khote Sikkay'), ('composer_name', 'Rahul Dev Burman'), ('citizenship', 'British India')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('s', 'http://www.wikidata.org/entity/Q3024718'), ('sname', 'The Wise Guys'), ('composer_name', 'François de Roubaix'), ('citizenship', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('s', 'http://www.wikidata.org/entity/Q8

In [145]:
# write your queries
# Q11424 --> 'film'
# Q172980 --> 'Western film'
# P86 --> 'composer' From the above query.
# P27 --> 'nationality' from the given data.
queryString = """
SELECT DISTINCT (COUNT(?s) as ?NumOfFilms) ?composer_name ?citizenship
WHERE {
   ?s ?p wd:Q172980;
   wdt:P86 ?cname.
   ?cname wdt:P27 ?coun.
    
    ?s sc:name ?film_name .
   ?cname sc:name ?composer_name.
   ?coun sc:name ?citizenship.
}
Group by ?citizenship ?composer_name
Order by desc(?NumOfFilms)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '82'), ('composer_name', 'Paul Sawtell'), ('citizenship', 'United States of America')]
[('NumOfFilms', '37'), ('composer_name', 'Cy Feuer'), ('citizenship', 'United States of America')]
[('NumOfFilms', '33'), ('composer_name', 'Raoul Kraushaar'), ('citizenship', 'United States of America')]
[('NumOfFilms', '30'), ('composer_name', 'Max Steiner'), ('citizenship', 'United States of America')]
[('NumOfFilms', '30'), ('composer_name', 'Max Steiner'), ('citizenship', 'Austria-Hungary')]
[('NumOfFilms', '25'), ('composer_name', 'Paul Dunlap'), ('citizenship', 'United States of America')]
[('NumOfFilms', '24'), ('composer_name', 'Victor Young'), ('citizenship', 'United States of America')]
[('NumOfFilms', '23'), ('composer_name', 'John Leipold'), ('citizenship', 'United States of America')]
[('NumOfFilms', '23'), ('composer_name', 'Roy Webb'), ('citizenship', 'United States of America')]
[('NumOfFilms', '22'), ('composer_name', 'David Buttolph'), ('citizenship', 'Unite

Final query for this task

In [7]:
# write your final query
queryString = """
SELECT DISTINCT ?coun ?composer_name  ?citizenship (COUNT(?s) as ?NumOfFilms) 
WHERE {
   ?s ?p wd:Q172980;
   wdt:P86 ?cname.
   ?cname wdt:P27 ?coun.
    
    ?s sc:name ?film_name .
   ?cname sc:name ?composer_name.
   ?coun sc:name ?citizenship.
}
Group by ?citizenship ?composer_name ?coun
Order by desc(?NumOfFilms)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Paul Sawtell'), ('citizenship', 'United States of America'), ('NumOfFilms', '82')]
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Cy Feuer'), ('citizenship', 'United States of America'), ('NumOfFilms', '37')]
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Raoul Kraushaar'), ('citizenship', 'United States of America'), ('NumOfFilms', '33')]
[('coun', 'http://www.wikidata.org/entity/Q28513'), ('composer_name', 'Max Steiner'), ('citizenship', 'Austria-Hungary'), ('NumOfFilms', '30')]
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Max Steiner'), ('citizenship', 'United States of America'), ('NumOfFilms', '30')]
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Paul Dunlap'), ('citizenship', 'United States of America'), ('NumOfFilms', '25')]
[('coun', 'http://www.wikidata.org/entity/Q30'), ('composer_name', 'Victor Young'), ('citizenshi

## Task 7
Consider the decades from 1961 to 1970, and from 2001 to 2010 and compare the total number of western films released in each one (the result set must be two couples decade and #western films of that decade).

In [9]:
#Grouping data wrt genre
# Q172980 --> 'Western film'
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?pname 
WHERE {
    ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?pname .
   #?Pdate sc:name ?objname. 
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
}
GROUP BY ?pname 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1'), ('pname', 'Tell Them Willie Boy Is Here')]
[('NumOfFilms', '1'), ('pname', 'The Desperado Trail')]
[('NumOfFilms', '1'), ('pname', 'Shalako')]
[('NumOfFilms', '1'), ('pname', 'Return of the Gunfighter')]
[('NumOfFilms', '1'), ('pname', 'Town Tamer')]
[('NumOfFilms', '2'), ('pname', 'Kid Rodelo')]
[('NumOfFilms', '1'), ('pname', 'Outlaw of Red River')]
[('NumOfFilms', '1'), ('pname', 'Day of the Evil Gun')]
[('NumOfFilms', '3'), ('pname', 'A Fistful of Dollars')]
[('NumOfFilms', '1'), ('pname', 'McLintock!')]
10


In [16]:
#Grouping data wrt genre
# Q172980 --> 'Western film'
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?film_name  ?Pdate
WHERE {{
 ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
   #?Pdate sc:name ?objname. 
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
   }
   UNION
{
    ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
   
   FILTER(?Pdate > "2001"^^xsd:gYear && ?Pdate < "2010"^^xsd:gYear)
   }
}
GROUP BY ?film_name ?Pdate

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1'), ('film_name', 'Gun Fight'), ('Pdate', '1961-05-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'A Taste of Death'), ('Pdate', '1968-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'Red Tomahawk'), ('Pdate', '1967-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'Outlaw of Red River'), ('Pdate', '1965-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'The Scalphunters'), ('Pdate', '1968-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'Cry Blood, Apache'), ('Pdate', '1970-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'A Fistful of Dollars'), ('Pdate', '1964-10-20T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'The Way West'), ('Pdate', '1967-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'The Magnificent Seven'), ('Pdate', '1961-05-03T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'Bullet for a Badman'), ('Pdate', '1964-01-01T00:00:00Z')]
[('NumOfFilms', '1'), ('film_name', 'The Elusive Avengers'), ('Pdate', 

In [22]:
#Grouping data wrt genre
# Q172980 --> 'Western film'
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?film_name  ?releaseyear
WHERE {{
 ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
   #?Pdate sc:name ?objname. 
   bind(STRBEFORE(str(?Pdate),'-' )as ?releaseyear).
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
   }
   UNION
{
   ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
    bind(STRBEFORE(str(?Pdate),'-' )as ?releaseyear).
   
   FILTER(?Pdate > "2001"^^xsd:gYear && ?Pdate < "2010"^^xsd:gYear)
   }
}
GROUP BY ?film_name ?releaseyear

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1'), ('film_name', 'The Drifting Avenger'), ('releaseyear', '1968')]
[('NumOfFilms', '1'), ('film_name', 'Kid Rodelo'), ('releaseyear', '1966')]
[('NumOfFilms', '1'), ('film_name', 'The Way West'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'Sartana Does Not Forgive'), ('releaseyear', '1968')]
[('NumOfFilms', '1'), ('film_name', 'Nebraska-Jim'), ('releaseyear', '1966')]
[('NumOfFilms', '1'), ('film_name', 'The Fastest Guitar Alive'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'Welcome to Hard Times'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'The Wild Westerners'), ('releaseyear', '1962')]
[('NumOfFilms', '1'), ('film_name', 'Gunfight at Comanche Creek'), ('releaseyear', '1963')]
[('NumOfFilms', '1'), ('film_name', 'Young Billy Young'), ('releaseyear', '1969')]
[('NumOfFilms', '1'), ('film_name', 'Challenge of the McKennas'), ('releaseyear', '1970')]
[('NumOfFilms', '1'), ('film_name', 'Death is serving'),

Final query for this task

In [23]:
# write your final query
queryString = """
SELECT (COUNT(?p) as ?NumOfFilms) ?film_name  ?releaseyear
WHERE {{
 ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
   #?Pdate sc:name ?objname. 
   bind(STRBEFORE(str(?Pdate),'-' )as ?releaseyear).
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
   }
   UNION
{
   ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate.
    ?p sc:name ?film_name .
    bind(STRBEFORE(str(?Pdate),'-' )as ?releaseyear).
   
   FILTER(?Pdate > "2001"^^xsd:gYear && ?Pdate < "2010"^^xsd:gYear)
   }
}
GROUP BY ?film_name ?releaseyear

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '1'), ('film_name', 'The Drifting Avenger'), ('releaseyear', '1968')]
[('NumOfFilms', '1'), ('film_name', 'Kid Rodelo'), ('releaseyear', '1966')]
[('NumOfFilms', '1'), ('film_name', 'The Way West'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'Sartana Does Not Forgive'), ('releaseyear', '1968')]
[('NumOfFilms', '1'), ('film_name', 'Nebraska-Jim'), ('releaseyear', '1966')]
[('NumOfFilms', '1'), ('film_name', 'The Fastest Guitar Alive'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'Welcome to Hard Times'), ('releaseyear', '1967')]
[('NumOfFilms', '1'), ('film_name', 'The Wild Westerners'), ('releaseyear', '1962')]
[('NumOfFilms', '1'), ('film_name', 'Gunfight at Comanche Creek'), ('releaseyear', '1963')]
[('NumOfFilms', '1'), ('film_name', 'Young Billy Young'), ('releaseyear', '1969')]
[('NumOfFilms', '1'), ('film_name', 'Challenge of the McKennas'), ('releaseyear', '1970')]
[('NumOfFilms', '1'), ('film_name', 'Death is serving'),

## Task 8
Consider the decades from 1961 to 1970, and from 2001 to 2010 and select only western films. Then, for each decade, compare the average cast members size per film (the result set must be two couples decade and average cast members size of that decade).

In [46]:
# write your queries
#Grouping data wrt genre
# Q172980 --> 'Western film'
# P161 --> 'cast member' (from task 4)
queryString = """
SELECT (AVG(?average_cast_member) as ?avg_per_film) 
WHERE {
    SELECT (Count(?cmember) AS ?average_cast_member) ?pname 
    WHERE {{
    ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate;
    wdt:P161 ?cmember.
    
    ?p sc:name ?pname .
    ?cmember sc:name ?cast_member.
   #?Pdate sc:name ?objname. 
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
   }
    UNION
{
   ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate;
    wdt:P161 ?cmember.
    
    ?p sc:name ?pname .
    ?cmember sc:name ?cast_member.
   #?Pdate sc:name ?objname.    
   
   FILTER(?Pdate > "2001"^^xsd:gYear && ?Pdate < "2010"^^xsd:gYear)
   }
   }
  
   
    GROUP BY ?pname
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('avg_per_film', '13.482758620689655')]
1


Final query for this task

In [47]:
# write your final query
queryString = """
SELECT (AVG(?average_cast_member) as ?avg_per_film) 
WHERE {
    SELECT (Count(?cmember) AS ?average_cast_member) ?pname 
    WHERE {{
    ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate;
    wdt:P161 ?cmember.
    
    ?p sc:name ?pname .
    ?cmember sc:name ?cast_member.
   #?Pdate sc:name ?objname. 
   
   FILTER(?Pdate > "1961"^^xsd:gYear && ?Pdate < "1970"^^xsd:gYear)
   }
    UNION
{
   ?p wdt:P31 wd:Q11424;
     wdt:P136 wd:Q172980; #those film who's genre is western film
    wdt:P577 ?Pdate;
    wdt:P161 ?cmember.
    
    ?p sc:name ?pname .
    ?cmember sc:name ?cast_member.
   #?Pdate sc:name ?objname.    
   
   FILTER(?Pdate > "2001"^^xsd:gYear && ?Pdate < "2010"^^xsd:gYear)
   }
   }
  
   
    GROUP BY ?pname
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('avg_per_film', '13.482758620689655')]
1
